## Install requirements ## 
We will need [pyaudio](http://people.csail.mit.edu/hubert/pyaudio/) package. For Windows it requires [VC++14 build tools](https://www.scivision.dev/python-windows-visual-c-14-required/), which can be obtained separately or as a part of other software. Install it before running `pip install pyaudio`. Or even better: unstall prebuilt wheel file [from here](https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio).

In [1]:
# install PyAudio 
# for details see http://people.csail.mit.edu/hubert/pyaudio/
# you will need Microsoft Visual C++ 14.0 build tools
# this is the easiest way to obtain - https://www.microsoft.com/en-US/download/details.aspx?id=48145

# but also you can alway look for a prebuilt wheel file here:
!pip install https://download.lfd.uci.edu/pythonlibs/s2jqpv5t/PyAudio-0.2.11-cp37-cp37m-win_amd64.whl

# linux
# !sudo apt-get install python-pyaudio python3-pyaudio

## Imports and setup ##

In [1]:
import pyaudio
import wave
import sys

In [2]:
# default settings for record and playback
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 22050

In [3]:
def play_stream(wave_stream, player):
    '''Takes a `wave.Wave_read` stream and plays'''
    global CHUNK
    print("Playing...", end="")
    out_stream = player.open(
                    output=True,
                    format=player.get_format_from_width(wave_stream.getsampwidth()),
                    channels=wave_stream.getnchannels(),
                    rate=wave_stream.getframerate()
    )
    data = wave_stream.readframes(CHUNK)
    while data:
        out_stream.write(data)
        data = wave_stream.readframes(CHUNK)
    out_stream.stop_stream()
    out_stream.close()
    print("played")

In [4]:
def record_to_file(filename, player, seconds=5):
    global FORMAT, CHANNELS, RATE
    RECORD_SECONDS = seconds

    stream = player.open(
                input=True,
                format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                frames_per_buffer=CHUNK)

    print("Start recording... ", end="")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)  # or you can here detect average magnitude and stop an silence
        frames.append(data)
    print("recorded", seconds, "second(s)")
    stream.stop_stream()
    stream.close()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(player.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [6]:
#sample = "datasets/sound/peterpiper.wav"
sample = "peterpiper.wav"
buffer = "buffer.wav"
player = pyaudio.PyAudio()
print("Recording devices:", player.get_device_count())
print("Default recorder:", player.get_default_input_device_info())


wf = wave.open(sample, 'rb')
print(type(wf))
play_stream(wf, player)

record_to_file(buffer, player)
wf = wave.open(buffer, 'rb')
play_stream(wf, player)

Recording devices: 4
Default recorder: {'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek High Defini', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
<class 'wave.Wave_read'>
Playing...played
Start recording... recorded 5 second(s)
Playing...played


In [7]:
player.terminate()

In [8]:
# https://realpython.com/playing-and-recording-sound-python/
!pip install sounddevice

In [9]:
import sounddevice as sd
from scipy.io.wavfile import write

seconds = 3  # Duration of recording

myrecording = sd.rec(int(seconds * RATE), samplerate=RATE, channels=CHANNELS)
sd.wait()  # Wait until recording is finished

write('output.wav', RATE, myrecording)  # Save as WAV file 